# Obnova obrazu

Nasnímaný obraz je běžně zatížen šumem z několika zrdojů. Těmito zdroji jsou např. šum vzniklý na elektrických senzorech, chyby kanálů, zrnitost atd. šum vzniklý od na senzorch nabo na kanálech je ve formě izolovaných variancí hodnot pixelů které nejsou mezi sebou korelované a které jsou v obraze viditelné jako výrazně odlišné hodnoty sousedních pixelů.

Na následujícím obrázku je blokovým diagramem vizualizovaný proces vzniku šumu a degradace obrazu např. rozmazáním.

![diagram 1](https://cw.fel.cvut.cz/wiki/_media/courses/b4m33dzo/labs/im_degradation.jpg)

ve skutečnosti je obraz degradován jak šumem tak degradační funkcí najednou, pro jedoduchost budeme uvažovat jen šum.

## Modely šumu

Existuje mnoho modelů šumu, v této hodině budeme uvažovat čtyři následující:

### Šum s rovnoměrným rozdělením
Šum s rovnoměrným rozdělením je generován z rozmezí  $low$ a $high$ dle předpisu:

$
\begin{equation}
p(z) = \begin{cases}
\frac{1}{high-low}, & \text{if } low \leq z \leq high \\
0, & \text{otherwise} \\
\end{cases}
\end{equation}
$

kde $p(z)$ je pravděpodobnost že pixel nabyde hodnoty šumu $z$.

### Gaussovský šum

Je generovaný pomocí normálního rozdělení dle předpisu:

$
\begin{equation}
p(z) = \frac{1}{ \sqrt{2\pi \sigma^2}} \; \exp \left( - \frac{(z - \mu)^2}{2\sigma^2} \right),
\end{equation}
$

kde $\mu$ je střední hodnota a $\sigma$ je směrodatná odchylka.

### Salt&Peper

Šum ve formě impulzů častu zůsoběný chybou A/D převodníku nebo chybou přenosu. Způsobuje že určité množství pixelů má 0 nebo maximální hodnotu.

Pravděpodobnost že pixel bude mít intenzitu $z$ je dána předpisem

$
\begin{equation}
p(z) = \begin{cases}
pPepper, & \text{for } z = 0 \text{ (pepper)} \\
pSalt, & \text{for } z = 2^n - 1 \text{ (salt)} \\
1-(pPepper+pSalt), & \text{for } 0 < z < 2^n - 1, \\
\end{cases}
\end{equation}
$

kde $n$ je počet bitů ve kterých je hodnota pixelů kódována (8). Pro vygenrování tohoto filtru využijte funkci pro generování náhodných čísel z rovnoměrného rozdělení.



### Šum s exponencíálním rozdělením

V případě exponencílního šumu je pravděpodobnost genrována předpisem:

$
\begin{equation}
p(z) = \begin{cases}
0, & z < 0 \\
\lambda \exp(-\lambda z), & z \geq \lambda \\
\end{cases}
\end{equation}
$


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img =  cv2.imread("data/tower.jpg")
fig = plt.figure(figsize= (10,10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB));

In [ ]:
def uniform_noise(img, low=-0.1, high=0.1):
    """
    Aplikace rovnomerneho sumu na obraz v rozmezi low az high
    
    Parameters
    ----------
    img : ndarray
        Obraz nacteny funkci z opencv
    low : float
        minimalni hodnota pro generovani rovnomerneho sumu
    high : float
        maximalni hodnota pro generovani rovnomerneho sumu
    
    Returns
    ----------
    tuple
        obraz se sumem, obraz sumu, aplikovane hodnoty sumu
    """
    
    noise_rnd = np.random.uniform(0, 1, img.shape) # Vygenerovani 3d matice nahodnych cisel v rozmezi 0, 1 o rozmerech obrazu nahodnych cisel 
    mask = np.where(np.logical_and(noise_rnd>0.5+low, noise_rnd<0.5+high)) #maska - indexy kde hodnot jsou v rozmezi low a high (+0.5)
    noise = np.zeros(img.shape)
    noise[mask] = (noise_rnd[mask] - 0.5) * 255 # Vygenerovani matice sumu pro aplikaci na obraz 
    img_noise = img + noise # aplikace matice sumu
    
    # normalizace noveho obrazu mezi 0 a 255
    img_noise[img_noise<0] = 0
    img_noise[img_noise>255] = 255
    
    return img_noise.astype(np.uint8), np.abs(noise).astype(np.uint8), noise_rnd.flatten() - 0.5

In [ ]:
fig = plt.figure(figsize= (40,6))
img_noise, noise, noise_raw = uniform_noise(img, -0.25, 0.25)
plt.subplot(141)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB));
plt.title('original')
plt.subplot(142)
plt.imshow(cv2.cvtColor(noise, cv2.COLOR_BGR2RGB));
plt.title('noise - uniform')
plt.subplot(143)
plt.imshow(cv2.cvtColor(img_noise, cv2.COLOR_BGR2RGB));
plt.title('original + noise')
plt.subplot(144)
plt.hist(noise_raw, density=True, bins=np.linspace(-0.25,0.25,100));
plt.title('noise histogram');

## Obnova obrazu

Pro obnovu degradovaného obrazu je nejprve potřeba odhadnou typ degradace. Budeme uvažovat pouze šum. Vizualizace procesu obnovy obrazu pomocí blokového schématu je na následujícím, obrázku:

![diagram 2](https://cw.fel.cvut.cz/wiki/_media/courses/b4m33dzo/labs/im_restoration.jpg)

## Odstranění šumu

Šum může být redukován pomocí statistických filtrů. Zde záleží jaký druh šumu je v obraze přítomen. Filtr je aplikován jako dané statistické operaci odpovídajcí konvoluční filtr. Předpokládejme obraz $g(s,t)$ kde $g$ a $s$ jsou souřadnice pixelu v obraze. Konvoluční filtr je pak $S_{xy}$.

### Mediánový filtr
Je vhodný pro redukci šumu typu Salt&Peper. Předpis pro výpočet nového obrazu je

$
\begin{equation}
\hat{f}(x,y) = \underset{(s,t) \in S_{xy}}{\mathrm{median}} \, g(s,t)
\end{equation}
$

### Průměrovací filtr

Je vhodný pro odstarněni Gaussovského šumu. Nový obraz vznikne dle předpisu

$
\begin{equation}
\hat{f}(x,y) = \frac{1}{mn}\sum_{(s,t) \in S_{xy}} g(s,t),
\end{equation}
$

kde $m$ a $n$ jsou rozměry konvolučního filtru.


** Pro mediánový a průměrovací filtr je možné využít následující funkce OpenCV2 https://docs.opencv.org/master/d4/d13/tutorial_py_filtering.html **


### MIN filtr
Odpovídá morfologické erozi.


$
\begin{equation}
\hat{f}(x,y) = \min_{(s,t) \in S_{xy}} g(s,t)
\end{equation}
$

### MAX filtr
Odpovídá morfologické dilataci.

$
\begin{equation}
\hat{f}(x,y) = \max_{(s,t) \in S_{xy}} g(s,t)
\end{equation}
$

** Pro MIN a MAX filtry je možné využít funkce OpenCV2 pro erozi a dilataci https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html **
